# 03 – BERT Model & Tweet Impact (Setup Instructions)

This notebook has **two parts**:

1. **Twitter entity sentiment model** using a Kaggle dataset (downloaded via `kagglehub`)
2. **Tweet impact on financial markets** using your `HODL Final Project` data

To run the whole notebook end-to-end:

1. Make sure you have already run **`01_data_processing.ipynb`** so it creates:
   - `data_clean/merged_dataset.parquet`
   - `data_clean/split_indices.npz`
   - `outputs/eval_utils.py`

2. Ensure the project folder **`HODL Final Project`** is in **your** Google Drive at:  
   `My Drive/HODL Final Project`

   - If this folder was shared with you, go to **"Shared with me"** in Google Drive,  
     right-click **`HODL Final Project` → "Add shortcut to Drive" → My Drive**.

3. If your folder lives somewhere else or has a different name,  
   **edit the `BASE_PATH` variable in the "Tweet impact on financial markets – setup" code cell.**

4. This notebook needs internet access to:
   - Download the Kaggle dataset via `kagglehub`
   - Download the BERT preset via `keras_hub`

5. Once the above is set, go to **Runtime → Run all** and authorize Drive access when prompted.


##Tweets sentiment analysis model

### Dataset

In [ ]:
# Environment setup (safe to re-run)
!pip install -q kagglehub keras-tuner keras-hub

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
import kagglehub
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

keras.utils.set_random_seed(42)

# Download latest version of the Kaggle dataset
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")
print("Path to dataset files:", path)

os.listdir(path)

Using Colab cache for faster access to the 'twitter-entity-sentiment-analysis' dataset.
Path to dataset files: /kaggle/input/twitter-entity-sentiment-analysis


In [ ]:
os.listdir(path)

['twitter_validation.csv', 'twitter_training.csv']

### Train / test / validation

In [ ]:
train_df = pd.read_csv(
    path + "/twitter_training.csv",
    header=None,
    names=["id", "entity", "sentiment", "text"],
    encoding="utf-8"
)

val_df = pd.read_csv(
    path + "/twitter_validation.csv",
    header=None,
    names=["id", "entity", "sentiment", "text"],
    encoding="utf-8"
)

In [ ]:
train_df.head()

,id,entity,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
print("Train:", train_df.shape)
print("Val:", val_df.shape)

Train: (74682, 4)
Val: (1000, 4)


In [ ]:
from sklearn.model_selection import train_test_split

val_df, test_df = train_test_split(
    val_df,
    test_size=0.5,
    random_state=42
)

print("Val:", val_df.shape, "Test:", test_df.shape)

Val: (500, 4) Test: (500, 4)


In [ ]:
label_dummies = pd.get_dummies(train_df["sentiment"], dtype="int")
class_names = label_dummies.columns.tolist()   # <-- save for later

y_train = label_dummies.to_numpy()

y_val = pd.get_dummies(val_df["sentiment"], dtype="int") \
            .reindex(columns=class_names, fill_value=0).to_numpy()

y_test = pd.get_dummies(test_df["sentiment"], dtype="int") \
             .reindex(columns=class_names, fill_value=0).to_numpy()

num_classes = y_train.shape[1]
print("Num classes:", num_classes)
print(y_train[:5])

Num classes: 4
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


###Standardize, Tokenize, Index and Encode

In [ ]:
from keras.layers import TextVectorization

max_tokens = 20000
output_sequence_length = 256

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="count",
)

# Fill any potential NaN values in the 'text' column with an empty string
text_vectorization.adapt(train_df["text"].astype(str).fillna(''))

vocabulary = text_vectorization.get_vocabulary()
print("Vocabulary size:", len(vocabulary))
print(vocabulary[:20])

Vocabulary size: 20000
['[UNK]', np.str_('the'), np.str_('i'), np.str_('to'), np.str_('and'), np.str_('a'), np.str_('of'), np.str_('is'), np.str_('for'), np.str_('in'), np.str_('this'), np.str_('it'), np.str_('you'), np.str_('on'), np.str_('my'), np.str_('that'), np.str_('with'), np.str_('game'), np.str_('so'), np.str_('me')]


In [ ]:
len(vocabulary)

20000

In [ ]:
print('Vocabulary:')
print(vocabulary)

Vocabulary:
['[UNK]', np.str_('the'), np.str_('i'), np.str_('to'), np.str_('and'), np.str_('a'), np.str_('of'), np.str_('is'), np.str_('for'), np.str_('in'), np.str_('this'), np.str_('it'), np.str_('you'), np.str_('on'), np.str_('my'), np.str_('that'), np.str_('with'), np.str_('game'), np.str_('so'), np.str_('me'), np.str_('have'), np.str_('just'), np.str_('but'), np.str_('not'), np.str_('are'), np.str_('be'), np.str_('all'), np.str_('was'), np.str_('at'), np.str_('like'), np.str_('out'), np.str_('from'), np.str_('its'), np.str_('your'), np.str_('now'), np.str_('2'), np.str_('get'), np.str_('we'), np.str_('as'), np.str_('im'), np.str_('they'), np.str_('has'), np.str_('if'), np.str_('one'), np.str_('do'), np.str_('good'), np.str_('about'), np.str_('can'), np.str_('play'), np.str_('will'), np.str_('no'), np.str_('an'), np.str_('really'), np.str_('when'), np.str_('love'), np.str_('new'), np.str_('up'), np.str_('what'), np.str_('more'), np.str_('time'), np.str_('by'), np.str_('how'), np.st

In [ ]:
print('Vocabulary:')
print(pd.DataFrame(vocabulary))

Vocabulary:
                          0
0                     [UNK]
1                       the
2                         i
3                        to
4                       and
...                     ...
19995      twitchtvosgshotz
19996  twitchtvorbitalshark
19997     twitchtvonlyakram
19998  twitchtvonenipwonder
19999     twitchtvoldseadog

[20000 rows x 1 columns]


In [ ]:
# Encode and decode an example sentence
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([0 0 1 ... 0 0 0], shape=(20000,), dtype=int64)


In [ ]:
X_train = text_vectorization(train_df["text"].astype(str).fillna(''))
X_val   = text_vectorization(val_df["text"].astype(str).fillna(''))
X_test  = text_vectorization(test_df["text"].astype(str).fillna(''))

print(X_train.shape)
X_train[:2]

(74682, 20000)


<tf.Tensor: shape=(2, 20000), dtype=int64, numpy=
array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 2, ..., 0, 0, 0]])>

### Baseline model

In [ ]:
inputs = keras.layers.Input(shape=(max_tokens,))
x = keras.layers.Dense(8, activation="relu")(inputs)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │       160,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,044 (625.17 KB)

 Trainable params: 160,044 (625.17 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32,
)

Epoch 1/5
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5679 - loss: 1.0842 - val_accuracy: 0.8840 - val_loss: 0.4187
Epoch 2/5
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8089 - loss: 0.5444 - val_accuracy: 0.9400 - val_loss: 0.2586
Epoch 3/5
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8603 - loss: 0.3883 - val_accuracy: 0.9460 - val_loss: 0.2087
Epoch 4/5
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8821 - loss: 0.3197 - val_accuracy: 0.9520 - val_loss: 0.1919
Epoch 5/5
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8937 - loss: 0.2823 - val_accuracy: 0.9540 - val_loss: 0.1866


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9585 - loss: 0.1482 
Test accuracy: 0.9539999961853027


In [ ]:
# Small function to test model behavior

def tweet_predict(text):
    vect = text_vectorization([text])
    preds = model.predict(vect)

    for label, p in zip(class_names, preds[0]):
        print(f"{label}: {p * 100:.2f}%")

In [ ]:
tweet_predict("Great stock, it's going to the moon!")
tweet_predict("The Tesla stock have been going really good lately, it's going to the moon!")
tweet_predict("The new CEO of Google is a disaster, I am shorting the company.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
Irrelevant: 31.28%
Negative: 1.63%
Neutral: 2.50%
Positive: 64.59%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Irrelevant: 12.17%
Negative: 0.15%
Neutral: 1.85%
Positive: 85.83%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Irrelevant: 0.07%
Negative: 88.01%
Neutral: 8.16%
Positive: 3.75%


### Optimizing hyperparameters

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
    inputs = keras.layers.Input(shape=(max_tokens,))
    x = inputs

    # 1, 2 or 3 hidden layers
    num_layers = hp.Int("num_layers", min_value=1, max_value=3, step=1)

    unit_options = [64, 128]   # 2 possible sizes

    for i in range(num_layers):
        units = hp.Choice(f"units_{i}", values=unit_options)
        x = keras.layers.Dense(units, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    # 2 learning-rate options
    lr = hp.Choice("lr", values=[1e-3, 3e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


In [ ]:
tuner = kt.GridSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    overwrite=True,
    directory="tweet_tuner",
    project_name="multi_hparam",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128], 'ordered': True}
lr (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0003], 'ordered': True}


In [ ]:
tuner.search(
    X_train,
    y_train,
    epochs=5,          # increase later
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose=1,
)

Trial 28 Complete [00h 00m 33s]
val_accuracy: 0.972000002861023

Best val_accuracy So Far: 0.9800000190734863
Total elapsed time: 00h 15m 19s


In [ ]:
best_model = tuner.get_best_models(1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
best_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,280,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,284,484 (4.90 MB)

 Trainable params: 1,284,484 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print("Best model – test accuracy:", test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - accuracy: 0.9753 - loss: 0.1286
Best model – test accuracy: 0.9760000109672546


##Tweet impact on financial markets

### Dataset

In [ ]:
from google.colab import drive
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Change if your folder is in a different place
BASE_PATH = '/content/drive/MyDrive/HODL Final Project'

DATA_CLEAN = f'{BASE_PATH}/data_clean'
OUTPUTS    = f'{BASE_PATH}/outputs'

# Work inside the project folder
os.chdir(BASE_PATH)

# Make sure we can import eval_utils
sys.path.append(OUTPUTS)

print(f"Working directory: {os.getcwd()}")
print(f"eval_utils exists: {os.path.exists(os.path.join(OUTPUTS, 'eval_utils.py'))}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working directory: /content/drive/MyDrive/HODL Final Project
eval_utils exists: True


In [ ]:
from eval_utils import evaluate_model, print_results

# Load processed dataset + splits created by 01_data_processing.ipynb
df = pd.read_parquet(f'{DATA_CLEAN}/merged_dataset.parquet')
splits = np.load(f'{DATA_CLEAN}/split_indices.npz')

In [ ]:
df.head()

,ticker,date_only,text,tweet_count,avg_tweet_len,total_chars,close,return,next_day_return,label
0,AAPL,2021-09-30,"I need 1,000 shares of $AAPL Returns over the ...",7,169.0,1183,138.390656,-0.009312,0.008128,1
1,AAPL,2021-10-01,The Results! #LottoFriday $AAPL 141P | .65 to ...,10,197.6,1976,139.515457,0.008128,-0.024606,0
2,AAPL,2021-10-04,$FB $AAPL $AMZN $MSFT $GOOGL *APPLE STOCK FALL...,10,192.5,1925,136.082550,-0.024606,0.014159,1
3,AAPL,2021-10-05,10/05/21 - Watch today's #MarketOutlook video ...,5,173.8,869,138.009277,0.014159,0.006307,1
4,AAPL,2021-10-06,Buying $SHIB is like buying $AAPL before Steve...,10,175.5,1755,138.879700,0.006307,0.009085,1


In [ ]:
df.shape

(2695, 10)

### Add a sentiment feature using the previous model

In [ ]:
texts_new = df["text"].astype(str)

X_new = text_vectorization(texts_new)

pred_probs = best_model.predict(X_new, batch_size=256, verbose=1)

pred_idx = pred_probs.argmax(axis=1)

df["sentiment"] = [class_names[i] for i in pred_idx]

df["sentiment_prob"] = pred_probs.max(axis=1)

df.head()

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 214ms/step


,ticker,date_only,text,tweet_count,avg_tweet_len,total_chars,close,return,next_day_return,label,sentiment,sentiment_prob
0,AAPL,2021-09-30,"I need 1,000 shares of $AAPL Returns over the ...",7,169.0,1183,138.390656,-0.009312,0.008128,1,Negative,0.999999
1,AAPL,2021-10-01,The Results! #LottoFriday $AAPL 141P | .65 to ...,10,197.6,1976,139.515457,0.008128,-0.024606,0,Neutral,1.000000
2,AAPL,2021-10-04,$FB $AAPL $AMZN $MSFT $GOOGL *APPLE STOCK FALL...,10,192.5,1925,136.082550,-0.024606,0.014159,1,Neutral,1.000000
3,AAPL,2021-10-05,10/05/21 - Watch today's #MarketOutlook video ...,5,173.8,869,138.009277,0.014159,0.006307,1,Neutral,1.000000
4,AAPL,2021-10-06,Buying $SHIB is like buying $AAPL before Steve...,10,175.5,1755,138.879700,0.006307,0.009085,1,Neutral,1.000000


### Train / test / split

In [ ]:
df_train = df.loc[splits['train_indices']]
df_val   = df.loc[splits['val_indices']]
df_test  = df.loc[splits['test_indices']]

In [ ]:
X_train = df_train['text'].values
y_train = df_train['label'].values

X_val   = df_val['text'].values
y_val   = df_val['label'].values

X_test  = df_test['text'].values
y_test  = df_test['label'].values


In [ ]:
numerical_feature_cols = [
    "tweet_count",
    "avg_tweet_len",
    "total_chars",
    # "close",
    # "return",
    "sentiment_prob"
]

categorical_feature_cols = [
    "sentiment"
]

# Extract numerical features
X_train_num = df_train[numerical_feature_cols].values
X_val_num   = df_val[numerical_feature_cols].values
X_test_num  = df_test[numerical_feature_cols].values

# Extract categorical features
X_train_cat = df_train[categorical_feature_cols].values
X_val_cat   = df_val[categorical_feature_cols].values
X_test_cat  = df_test[categorical_feature_cols].values

X_train_text = df_train['text'].values
X_val_text   = df_val['text'].values
X_test_text  = df_test['text'].values

y_train = df_train['label'].values.astype("int32")
y_val   = df_val['label'].values.astype("int32")
y_test  = df_test['label'].values.astype("int32")

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Scale numerical features
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_val_num_scaled   = scaler.transform(X_val_num)
X_test_num_scaled  = scaler.transform(X_test_num)

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train_cat)
X_val_cat_encoded   = encoder.transform(X_val_cat)
X_test_cat_encoded  = encoder.transform(X_test_cat)

# Concatenate all meta-features
X_train_meta = np.concatenate([X_train_num_scaled, X_train_cat_encoded], axis=1)
X_val_meta   = np.concatenate([X_val_num_scaled, X_val_cat_encoded], axis=1)
X_test_meta  = np.concatenate([X_test_num_scaled, X_test_cat_encoded], axis=1)

print("X_train_meta shape:", X_train_meta.shape)
print("X_val_meta shape:", X_val_meta.shape)
print("X_test_meta shape:", X_test_meta.shape)

X_train_meta shape: (1909, 8)
X_val_meta shape: (395, 8)
X_test_meta shape: (391, 8)


In [ ]:
# # ==== Downsample for fast experimentation ====
# N_train = 140
# N_val   = 30
# N_test  = 30

# df_train = df_train.sample(N_train, random_state=42)
# df_val   = df_val.sample(N_val,   random_state=42)
# df_test  = df_test.sample(N_test, random_state=42)

# X_train = df_train["text"].values
# y_train = df_train["label"].values.astype("int32")

# X_val   = df_val["text"].values
# y_val   = df_val["label"].values.astype("int32")

# X_test  = df_test["text"].values
# y_test  = df_test["label"].values.astype("int32")

# print(df_train.shape, df_val.shape, df_test.shape)

### Use BERT backbone

In [ ]:
import keras_hub

bert = keras_hub.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    activation="softmax",
    num_classes=2,      # binary classification
)

In [ ]:
train_tok = bert.preprocessor(list(X_train))
train_bert_output = bert.backbone.predict(
    train_tok,
    batch_size=32,
    verbose=1
)["sequence_output"][:, 0, :]

60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 200ms/step


In [ ]:
val_tok = bert.preprocessor(list(X_val))
val_bert_output = bert.backbone.predict(
    val_tok,
    batch_size=32,
    verbose=1
)["sequence_output"][:, 0, :]

13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 453ms/step


In [ ]:
test_tok = bert.preprocessor(list(X_test))
test_bert_output = bert.backbone.predict(
    test_tok,
    batch_size=32,
    verbose=1
)["sequence_output"][:, 0, :]   # (n_test, 768)

13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 652ms/step


In [ ]:
X_train_all = np.concatenate([train_bert_output, X_train_meta], axis=1)
X_val_all   = np.concatenate([val_bert_output,   X_val_meta], axis=1)
X_test_all  = np.concatenate([test_bert_output,  X_test_meta], axis=1)

print(X_train_all.shape)

(1909, 776)


### MLP head on top of CLS embedding

In [ ]:
import keras

In [ ]:
inputs = keras.layers.Input(shape=(768,))

x = keras.layers.Dense(128, activation="relu")(inputs)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)

In [ ]:
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

bert_head = keras.Model(inputs, outputs)
bert_head.summary()

bert_head.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115,073 (449.50 KB)

 Trainable params: 115,073 (449.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = bert_head.fit(
    train_bert_output,
    y_train,
    validation_data=(val_bert_output, y_val),
    epochs=10,
    batch_size=32,
    verbose=1,
)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.5103 - loss: 0.7149 - val_accuracy: 0.4937 - val_loss: 0.7108
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5108 - loss: 0.6944 - val_accuracy: 0.4911 - val_loss: 0.7281
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5079 - loss: 0.6964 - val_accuracy: 0.4835 - val_loss: 0.7311
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5351 - loss: 0.6907 - val_accuracy: 0.4911 - val_loss: 0.7307
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5463 - loss: 0.6793 - val_accuracy: 0.5013 - val_loss: 0.7331
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5684 - loss: 0.6768 - val_accuracy: 0.4962 - val_loss: 0.7473
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5564 - loss: 0.6811 - val_accuracy: 0.4886 - val_loss: 0.7464
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5786 - loss: 0.6679 - val_accuracy: 0.4987 - val_loss

In [ ]:
y_prob = bert_head.predict(test_bert_output).ravel()
y_pred = (y_prob >= 0.5).astype("int32")

results = evaluate_model(y_test, y_pred, y_prob)
print_results(results, model_name="BERT MLP Head")

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
BERT MLP Head: 48.08% acc, 0.283 F1, 0.478 AUC
